In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load CSV data
csv_path = 'gs://storage_for_all/DataSets/FacialEmotionRecognitionImageDataset_v1/data.csv'
df = pd.read_csv(csv_path)
df = df.drop(df.columns[0], axis=1)

df = df.loc[:35]# Adjust this part to ensure correct path handling:

# Adjust paths
df['path'] = df['path'].apply(lambda x: f"gs://storage_for_all/DataSets/FacialEmotionRecognitionImageDataset_v1/dataset/{x.split('/')[-2]}/{x.split('/')[-1]}")

# Label encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['label'])
one_hot_encoded_labels = to_categorical(integer_encoded)

# Dataset preparation function
def load_image(file_path, label):
    """Load and preprocess images from file paths."""
    try:
        image_data = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image_data, channels=3)
        image = tf.image.resize(image, [224, 224])  # Resize as needed
        return image, label
    except tf.errors.NotFoundError:
        print(f"Failed to load image at: {file_path}")
        return None, label  # Handle missing files gracefully

# Create TensorFlow datasets
full_dataset = tf.data.Dataset.from_tensor_slices((df['path'].tolist(), one_hot_encoded_labels))
full_dataset = full_dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
full_dataset = full_dataset.filter(lambda x, y: x is not None and y is not None)
full_dataset = full_dataset.batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Split dataset into training and validation
dataset_size = len(df)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size

train_dataset = full_dataset.take(train_size)
val_dataset = full_dataset.skip(train_size).take(val_size)

# Model definition
def tf_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), padding='same', input_shape=(224, 224, 3)),
        layers.LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128),
        layers.LeakyReLU(alpha=0.1),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

num_classes = len(label_encoder.classes_)  # Number of unique classes
model = tf_model(num_classes)

# Model training
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    use_multiprocessing=False,
    workers=1
)

# Save the model
model_save_path = "/models/emotion_detection_cnn_v1"
model.save(model_save_path)

# Evaluate the model
test_loss, test_acc = model.evaluate(val_dataset)
print("Test accuracy:", test_acc)

# Prediction function
def predict_emotion(image_path):
    img, _ = load_image(image_path, None)
    img = tf.expand_dims(img, 0)  # Make batch of 1
    predictions = model.predict(img)
    predicted_class = tf.argmax(predictions[0]).numpy()
    return label_encoder.inverse_transform([predicted_class])[0]  # Assuming labels are ordered

# Example usage
predicted_emotion = predict_emotion('gs://storage_for_all/DataSets/FacialEmotionRecognitionImageDataset_v1/dataset/Surprise/1bd930d6a1c717c11be33db74823f661cb53f86cbba1d3d1f336cdf9~12fffff.jpg')
print("Predicted Emotion:", predicted_emotion)


2024-08-03 03:23:27.719219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-03 03:23:28.990131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-08-03 03:23:28.990329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2024-08-03 03:23:30.611106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-08-03 03:23:30.611157: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-08-03 03:23:30.611192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (google-gemini-api): /proc/driver/nvidia/version does not exist
2024-08-03 03:23:30.611622: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with th

Epoch 1/10
     11/Unknown - 29s 2s/step - loss: 0.9414 - accuracy: 0.9205


KeyboardInterrupt

